In [1]:
from pyspark.sql import SparkSession
from pyspark import SparkConf

sparkSession = SparkSession.builder.config(conf= SparkConf() \
                        .setAppName('DataFrame') \
                        .setMaster('local[4]')) \
                        .enableHiveSupport().getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/01/01 14:20:53 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/01/01 14:20:53 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
23/01/01 14:20:53 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
23/01/01 14:20:53 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.
23/01/01 14:20:53 WARN Utils: Service 'SparkUI' could not bind on port 4044. Attempting port 4045.
23/01/01 14:20:53 WARN Utils: Service 'SparkUI' could not bind on port 4045. Attempting port 4046.
23/01/01 14:20:53 WARN Utils: Service 'SparkUI' could not bind on port 4046. Attempting port 4047.


In [2]:
df = sparkSession.read.csv('/Datasets/data.csv', header=True, inferSchema=True)

In [3]:
df.show(5)

+-----+------------+---+------------+----------+-------+---------+---+--------+--------+----+------+-------+------------+-------------+--------------+-------+-------------+-------+--------------------+---+
|USMER|MEDICAL_UNIT|SEX|PATIENT_TYPE| DATE_DIED|INTUBED|PNEUMONIA|AGE|PREGNANT|DIABETES|COPD|ASTHMA|INMSUPR|HIPERTENSION|OTHER_DISEASE|CARDIOVASCULAR|OBESITY|RENAL_CHRONIC|TOBACCO|CLASIFFICATION_FINAL|ICU|
+-----+------------+---+------------+----------+-------+---------+---+--------+--------+----+------+-------+------------+-------------+--------------+-------+-------------+-------+--------------------+---+
|    2|           1|  1|           1|03/05/2020|     97|        1| 65|       2|       2|   2|     2|      2|           1|            2|             2|      2|            2|      2|                   3| 97|
|    2|           1|  2|           1|03/06/2020|     97|        1| 72|      97|       2|   2|     2|      2|           1|            2|             2|      1|            1|    

#### pyspark.sql.Column.alias

In [35]:
df2 = df.select(df.AGE.alias('age2'))

In [10]:
df2.show(5)

+----+
|age2|
+----+
|  65|
|  72|
|  55|
|  53|
|  68|
+----+
only showing top 5 rows



#### pyspark.sql.Column.asc

In [17]:
df2.orderBy(df2.age2.asc()).show(5)

+----+
|age2|
+----+
|   0|
|   0|
|   0|
|   0|
|   0|
+----+
only showing top 5 rows



In [19]:
df.select('USMER','AGE').orderBy(df.AGE.asc()).show(5)

+-----+---+
|USMER|AGE|
+-----+---+
|    2|  0|
|    1|  0|
|    1|  0|
|    2|  0|
|    2|  0|
+-----+---+
only showing top 5 rows



#### pyspark.sql.Column.asc_nulls_first

In [38]:
df5 = sparkSession.createDataFrame([('Tom', 80), (None, 60), ('Alice', None)], ["name", "height"])

In [22]:
df5.orderBy(df5.height.asc_nulls_first()).show()

+-----+------+
| name|height|
+-----+------+
|Alice|  null|
| null|    60|
|  Tom|    80|
+-----+------+



In [23]:
df5.orderBy(df5.height.asc_nulls_last()).show()

+-----+------+
| name|height|
+-----+------+
| null|    60|
|  Tom|    80|
|Alice|  null|
+-----+------+



#### pyspark.sql.Column.astype

In [26]:
df2 = df2.select(df2.age2.astype('String'))
df2.printSchema()

root
 |-- age2: string (nullable = true)



In [33]:
df2 = df2.select(df2.age2.astype('Integer'))
df2.printSchema()

TypeError: 'Column' object is not callable

#### pyspark.sql.Column.between

In [28]:
df5.select(df5.name, df5.height.between(50, 70)).show()

+-----+-----------------------------------+
| name|((height >= 50) AND (height <= 70))|
+-----+-----------------------------------+
|  Tom|                              false|
| null|                               true|
|Alice|                               null|
+-----+-----------------------------------+



#### pyspark.sql.Column.cast

In [75]:
from pyspark.sql.types import *
from pyspark.sql.functions  import *

In [36]:
df2 = df2.select(df2.age2.cast(IntegerType()))
df2.printSchema()

root
 |-- age2: integer (nullable = true)



In [37]:
df2 = df2.select(df2.age2.cast(StringType()))
df2.printSchema()

root
 |-- age2: string (nullable = true)



In [39]:
df5.select('*').filter(df5.name.contains('A')).show()

+-----+------+
| name|height|
+-----+------+
|Alice|  null|
+-----+------+



In [41]:
df5.select('*').filter(df5.name.contains('n')).show()

+----+------+
|name|height|
+----+------+
+----+------+



#### pyspark.sql.Column.desc

In [42]:
df5.select('*').orderBy(df5.height.desc()).show()

+-----+------+
| name|height|
+-----+------+
|  Tom|    80|
| null|    60|
|Alice|  null|
+-----+------+



In [43]:
df5.select('*').orderBy(df5.height.desc_nulls_first()).show()

+-----+------+
| name|height|
+-----+------+
|Alice|  null|
|  Tom|    80|
| null|    60|
+-----+------+



In [44]:
df5.select('*').orderBy(df5.height.desc_nulls_last()).show()

+-----+------+
| name|height|
+-----+------+
|  Tom|    80|
| null|    60|
|Alice|  null|
+-----+------+



#### pyspark.sql.Column.dropFields 

In [47]:
data = [
    Row(name="Alex", age=20, friend=Row(name="Bob",age=30,height=150)),
    Row(name="Cathy", age=40, friend=Row(name="Doge",age=40,height=180))
]

df6 = sparkSession.createDataFrame(data)
df6.show()          

+-----+---+---------------+
| name|age|         friend|
+-----+---+---------------+
| Alex| 20| {Bob, 30, 150}|
|Cathy| 40|{Doge, 40, 180}|
+-----+---+---------------+



In [48]:
df6.withColumn('new', df6.friend.dropFields('age')).show()


+-----+---+---------------+-----------+
| name|age|         friend|        new|
+-----+---+---------------+-----------+
| Alex| 20| {Bob, 30, 150}| {Bob, 150}|
|Cathy| 40|{Doge, 40, 180}|{Doge, 180}|
+-----+---+---------------+-----------+



In [49]:
df6.withColumn('new', df6.friend.dropFields('age','name')).show()

+-----+---+---------------+-----+
| name|age|         friend|  new|
+-----+---+---------------+-----+
| Alex| 20| {Bob, 30, 150}|{150}|
|Cathy| 40|{Doge, 40, 180}|{180}|
+-----+---+---------------+-----+



#### pyspark.sql.Column.endswith

In [52]:
df6.filter(df6.name.endswith('x')).show()

+----+---+--------------+
|name|age|        friend|
+----+---+--------------+
|Alex| 20|{Bob, 30, 150}|
+----+---+--------------+



In [53]:
df6.filter(df6.name.endswith('y')).show()

+-----+---+---------------+
| name|age|         friend|
+-----+---+---------------+
|Cathy| 40|{Doge, 40, 180}|
+-----+---+---------------+



#### pyspark.sql.Column.getField

In [55]:
df6.select(df6.friend.getField('name')).show()

+-----------+
|friend.name|
+-----------+
|        Bob|
|       Doge|
+-----------+



In [57]:
df6.select(df6.friend.getField('name'),df6.friend.getField('age')).show()

+-----------+----------+
|friend.name|friend.age|
+-----------+----------+
|        Bob|        30|
|       Doge|        40|
+-----------+----------+



In [58]:
df6.select(df6.friend.age).show()

+----------+
|friend.age|
+----------+
|        30|
|        40|
+----------+



#### pyspark.sql.Column.getItem

In [59]:
df7 = sparkSession.createDataFrame([([1, 2], {"key": "value"})], ["l", "d"])
df7.show()

+------+--------------+
|     l|             d|
+------+--------------+
|[1, 2]|{key -> value}|
+------+--------------+



In [63]:
df7.select(df7.l.getItem(1), df7.d.getItem('key')).show()

+----+------+
|l[1]|d[key]|
+----+------+
|   2| value|
+----+------+



#### pyspark.sql.Column.ilike
SQL ILIKE expression (case insensitive LIKE). Returns a boolean Column based on a case insensitive match.

In [66]:
df6.select('*').filter(df6.name.ilike('%x')).show()

+----+---+--------------+
|name|age|        friend|
+----+---+--------------+
|Alex| 20|{Bob, 30, 150}|
+----+---+--------------+



In [68]:
df6.select('*').filter(df6.name.ilike('%y')).show()

+-----+---+---------------+
| name|age|         friend|
+-----+---+---------------+
|Cathy| 40|{Doge, 40, 180}|
+-----+---+---------------+



#### pyspark.sql.Column.isNotNull

In [69]:
df5.select('*').filter(df5.height.isNotNull()).show()

+----+------+
|name|height|
+----+------+
| Tom|    80|
|null|    60|
+----+------+



In [70]:
df5.select('*').filter(df5.height.isNull()).show()

+-----+------+
| name|height|
+-----+------+
|Alice|  null|
+-----+------+



#### pyspark.sql.Column.isin

In [71]:
df5.select('*').filter(df5.name.isin('Alice','Nu')).show()

+-----+------+
| name|height|
+-----+------+
|Alice|  null|
+-----+------+



In [72]:
df5.select('*').filter(df5.name.isin('Alice','Tom')).show()

+-----+------+
| name|height|
+-----+------+
|  Tom|    80|
|Alice|  null|
+-----+------+



#### pyspark.sql.Column.like

In [73]:
df5.select('*').filter(df5.name.like('Al%')).show()

+-----+------+
| name|height|
+-----+------+
|Alice|  null|
+-----+------+



#### pyspark.sql.Column.name

In [74]:
df5.select(df5.name.name('Name')).show()


+-----+
| Name|
+-----+
|  Tom|
| null|
|Alice|
+-----+



#### pyspark.sql.Column.otherwise

In [78]:
df.select(df.AGE, when(df.SEX == 1, 'M').otherwise('F').alias('Gender')).show(5)

+---+------+
|AGE|Gender|
+---+------+
| 65|     M|
| 72|     F|
| 55|     F|
| 53|     M|
| 68|     F|
+---+------+
only showing top 5 rows



#### pyspark.sql.Column.rlike

In [82]:
df6.select('*').filter(df6.name.rlike('lex$')).show()

+----+---+--------------+
|name|age|        friend|
+----+---+--------------+
|Alex| 20|{Bob, 30, 150}|
+----+---+--------------+



#### pyspark.sql.Column.startswith

In [83]:
df5.select('*').filter(df5.name.startswith('A')).show()

+-----+------+
| name|height|
+-----+------+
|Alice|  null|
+-----+------+



#### pyspark.sql.Column.substr

In [84]:
df5.select(df5.name.substr(1,3).alias('new')).show()

+----+
| new|
+----+
| Tom|
|null|
| Ali|
+----+



In [86]:
df5.select(df5.name.substr(2,3).alias('new')).show()

+----+
| new|
+----+
|  om|
|null|
| lic|
+----+



23/01/02 11:21:12 WARN DataStreamer: Exception for BP-241050457-127.0.0.1-1670841776195:blk_1073762840_22041
java.net.SocketTimeoutException: 65000 millis timeout while waiting for channel to be ready for read. ch : java.nio.channels.SocketChannel[connected local=/127.0.0.1:48080 remote=/127.0.0.1:9866]
	at org.apache.hadoop.net.SocketIOWithTimeout.doIO(SocketIOWithTimeout.java:163)
	at org.apache.hadoop.net.SocketInputStream.read(SocketInputStream.java:161)
	at org.apache.hadoop.net.SocketInputStream.read(SocketInputStream.java:131)
	at org.apache.hadoop.net.SocketInputStream.read(SocketInputStream.java:118)
	at java.io.FilterInputStream.read(FilterInputStream.java:83)
	at java.io.FilterInputStream.read(FilterInputStream.java:83)
	at org.apache.hadoop.hdfs.protocolPB.PBHelperClient.vintPrefixed(PBHelperClient.java:519)
	at org.apache.hadoop.hdfs.protocol.datatransfer.PipelineAck.readFields(PipelineAck.java:213)
	at org.apache.hadoop.hdfs.DataStreamer$ResponseProcessor.run(DataStreamer